In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import random

In [2]:
data97=pd.read_csv('data/97.csv')
data105=pd.read_csv('data/105.csv')
data118=pd.read_csv('data/118.csv')
data130=pd.read_csv('data/130.csv')
data169=pd.read_csv('data/169.csv')
data185=pd.read_csv('data/185.csv')
data197=pd.read_csv('data/197.csv')
data209=pd.read_csv('data/209.csv')
data222=pd.read_csv('data/222.csv')
data234=pd.read_csv('data/234.csv')

In [3]:
for i in (data97,data105,data118,data130,data169,data185,data197,data209,data222,data234):
    print(i.shape)
    print(i.isnull().any())

(243938, 4)
Unnamed: 0      False
X097_DE_time    False
X097_FE_time    False
X097RPM          True
dtype: bool
(121265, 5)
Unnamed: 0      False
X105_DE_time    False
X105_FE_time    False
X105_BA_time    False
X105RPM          True
dtype: bool
(122571, 5)
Unnamed: 0      False
X118_DE_time    False
X118_FE_time    False
X118_BA_time    False
X118RPM          True
dtype: bool
(121991, 5)
Unnamed: 0      False
X130_DE_time    False
X130_FE_time    False
X130_BA_time    False
X130RPM          True
dtype: bool
(121846, 5)
Unnamed: 0      False
X169_DE_time    False
X169_FE_time    False
X169_BA_time    False
X169RPM          True
dtype: bool
(121846, 5)
Unnamed: 0      False
X185_DE_time    False
X185_FE_time    False
X185_BA_time    False
X185RPM          True
dtype: bool
(121846, 5)
Unnamed: 0      False
X197_DE_time    False
X197_FE_time    False
X197_BA_time    False
X197RPM          True
dtype: bool
(122136, 5)
Unnamed: 0      False
X209_DE_time    False
X209_FE_time    False
X209_B

删除多余的列，一个疑问，关于加速度，为什么normal数据没有这个feature

In [4]:
data97=data97.drop(['X097RPM','Unnamed: 0'],axis=1)

In [5]:
data105=data105.drop(['X105RPM','Unnamed: 0', 'X105_BA_time'],axis=1)

In [6]:
data118=data118.drop(['X118RPM','Unnamed: 0', 'X118_BA_time'],axis=1)

In [7]:
data130=data130.drop(['X130RPM','Unnamed: 0', 'X130_BA_time'],axis=1)

In [8]:
data169=data169.drop(['X169RPM','Unnamed: 0', 'X169_BA_time'],axis=1)

In [9]:
data185=data185.drop(['X185RPM','Unnamed: 0', 'X185_BA_time'],axis=1)

In [10]:
data197=data197.drop(['X197RPM','Unnamed: 0', 'X197_BA_time'],axis=1)

In [11]:
data209=data209.drop(['X209RPM','Unnamed: 0', 'X209_BA_time'],axis=1)

In [12]:
data222=data222.drop(['X222RPM','Unnamed: 0', 'X222_BA_time'],axis=1)

In [13]:
data234=data234.drop(['X234RPM','Unnamed: 0', 'X234_BA_time'],axis=1)

In [14]:
data97.head()

,X097_DE_time,X097_FE_time
0,0.053197,0.145667
1,0.088662,0.097796
2,0.099718,0.054856
3,0.058621,0.036982
4,-0.004590,0.054445


In [15]:
data105.head()

,X105_DE_time,X105_FE_time
0,-0.083004,-0.402075
1,-0.195734,-0.004725
2,0.233419,-0.106631
3,0.103958,-0.074169
4,-0.181115,0.208947


In [16]:
for i in (data97,data105,data118,data130,data169,data185,data197,data209,data222,data234):
    print(i.shape)
    print(i.isnull().any())

(243938, 2)
X097_DE_time    False
X097_FE_time    False
dtype: bool
(121265, 2)
X105_DE_time    False
X105_FE_time    False
dtype: bool
(122571, 2)
X118_DE_time    False
X118_FE_time    False
dtype: bool
(121991, 2)
X130_DE_time    False
X130_FE_time    False
dtype: bool
(121846, 2)
X169_DE_time    False
X169_FE_time    False
dtype: bool
(121846, 2)
X185_DE_time    False
X185_FE_time    False
dtype: bool
(121846, 2)
X197_DE_time    False
X197_FE_time    False
dtype: bool
(122136, 2)
X209_DE_time    False
X209_FE_time    False
dtype: bool
(121991, 2)
X222_DE_time    False
X222_FE_time    False
dtype: bool
(122426, 2)
X234_DE_time    False
X234_FE_time    False
dtype: bool


In [17]:
# target = np.zeros((1, 10))
# target[0][9] = 1

In [18]:
# print(target)

In [19]:
# np.concatenate([target for i in range(10)])

In [20]:
# a = []
# for i in range(10):
#     a.append(target)
# print(a)

In [21]:
# train_data97 = []
# test_data97 = []
# n_step = 400    #时序数据的长度
# gap =400 #产生时序数据样本的间隔，每隔400个取400个作为时序数据
# n = data97.shape[0] // gap #生成时序样本的个数
# for i in range(n-1):
#     train_data97.append(data97.values[i * gap :(i + 1) * gap])
#     #print(len(train_data))
# train_data97 = np.array(train_data97)
# print(train_data97.shape)
# # print(len(train_data97))
# train_data97[[0, 3]].shape

In [22]:
def processing(data, n_step, gap, sample_number, test_ratio, label):
    all_data = []
    all_label = []
    target = np.zeros((1, 10))
    target[0][label] = 1
    n = data.shape[0] // gap #生成时序样本的个数
    for i in range(n-1):
        all_data.append(data.values[i * gap :(i + 1) * gap])
    all_data = np.array(all_data)
    idx = random.sample(range(all_data.shape[0]), sample_number)#返回了索引list
    test_number = int(sample_number * test_ratio)
    test_data = all_data[idx[:test_number]]#取test_number个样本作为测试集
    train_data = all_data[idx[test_number:]]
    train_label = np.concatenate([target for i in range(train_data.shape[0])])
    test_label = np.concatenate([target for i in range(test_data.shape[0])])
#     print('Train shape: {}'.format(train_data.shape))
#     print('Test shape: {}'.format(test_data.shape))
    return train_data, test_data, train_label, test_label


In [23]:
# a = random.sample(range(100), 10)
# print(a)
# print(a[:2])

In [24]:
train_sets = []
test_sets = []
train_label = []
test_label = []
data_list = [data97, data105, data118, data130, data169, data185, data197, data209, data222, data234]

n_step = 400
gap = 400
sample_number = 300
test_ratio = 0.1



for i, data in enumerate(data_list):
    a, b, c, d = processing(
        data, 
        n_step = n_step, 
        gap = gap, 
        sample_number = sample_number, 
        test_ratio = test_ratio,
        label = i
    )
    train_sets.append(a)
    test_sets.append(b)
    train_label.append(c)
    test_label.append(d)
train_sets = np.concatenate(train_sets)
test_sets = np.concatenate(test_sets)
train_label = np.concatenate(train_label)
test_label = np.concatenate(test_label)

In [25]:
state =np.random.get_state()
np.random.shuffle(train_sets)
np.random.set_state(state)
np.random.shuffle(train_label)


In [26]:
print(train_sets.shape)
print(test_sets.shape)
print(train_label.shape)
print(test_label.shape)

(2700, 400, 2)
(300, 400, 2)
(2700, 10)
(300, 10)


In [36]:
import gc
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.optimizers import Adam

In [37]:
# # create and fit the LSTM network
# model = Sequential()
# model.add(LSTM(32, 
#                return_sequences=True,
#                input_shape=(400, 2),
#                activation='relu')
#          )
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=True, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(units=10))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [38]:
model = Sequential([
    GRU(32, input_dim = 2, input_length = 400, return_sequences = True),
    GRU(16, return_sequences = False),
    # Dropout(0.5),
    Dense(10),
    Activation('softmax')
])

Adam_op = Adam(lr=0.001, decay=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=Adam_op, metrics=['accuracy'])
model.summary()



C:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(32, return_sequences=True, input_shape=(400, 2))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 400, 32)           3360      
_________________________________________________________________
gru_8 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                170       
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 5,882
Trainable params: 5,882
Non-trainable params: 0
_________________________________________________________________


In [39]:
print('Training ------------')
model.fit(train_sets, train_label, epochs=500, batch_size=100, validation_data=(test_sets, test_label), shuffle=True)  

Training ------------
Train on 2700 samples, validate on 300 samples
Epoch 1/500
2700/2700 [==============================] - 24s 9ms/step - loss: 2.3014 - acc: 0.0856 - val_loss: 2.2982 - val_acc: 0.1200
Epoch 2/500
2700/2700 [==============================] - 22s 8ms/step - loss: 2.2944 - acc: 0.1567 - val_loss: 2.2861 - val_acc: 0.1833
Epoch 3/500
2700/2700 [==============================] - 22s 8ms/step - loss: 2.2405 - acc: 0.1774 - val_loss: 1.9752 - val_acc: 0.2000
Epoch 4/500
2700/2700 [==============================] - 23s 8ms/step - loss: 1.8596 - acc: 0.2011 - val_loss: 1.8188 - val_acc: 0.2100
Epoch 5/500
2700/2700 [==============================] - 23s 8ms/step - loss: 1.7800 - acc: 0.2070 - val_loss: 1.7828 - val_acc: 0.2233
Epoch 6/500
2700/2700 [==============================] - 22s 8ms/step - loss: 1.7568 - acc: 0.2059 - val_loss: 1.7690 - val_acc: 0.2900
Epoch 7/500
2700/2700 [==============================] - 22s 8ms/step - loss: 1.7526 - acc: 0.2548 - val_loss: 1.77